In [23]:
# 以下程序为预测离散化之后的sin函数
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn

# 加载matplotlib工具包，使用该工具包可以对预测的sin函数曲线进行绘图
#import matplotlib as mpl
from tensorflow.contrib.learn.python.learn.estimators.estimator import SKCompat
#mpl.use('Agg')
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
learn = tf.contrib.learn
HIDDEN_SIZE = 30  # Lstm中隐藏节点的个数
NUM_LAYERS = 2  # LSTM的层数
TIMESTEPS = 10  # 循环神经网络的截断长度
TRAINING_STEPS = 10000  # 训练轮数
BATCH_SIZE = 32  # batch大小

TRAINING_EXAMPLES = 10000  # 训练数据个数
TESTING_EXAMPLES = 1000  # 测试数据个数
SAMPLE_GAP = 0.01  # 采样间隔
# 定义生成正弦数据的函数
def generate_data(seq):
    X = []
    Y = []
    # 序列的第i项和后面的TIMESTEPS-1项合在一起作为输入;第i+TIMESTEPS项作为输出
    # 即用sin函数前面的TIMESTPES个点的信息，预测第i+TIMESTEPS个点的函数值
    for i in range(len(seq) - TIMESTEPS - 1):
        X.append([seq[i:i + TIMESTEPS]])
        Y.append([seq[i + TIMESTEPS]])
    return np.array(X, dtype=np.float32), np.array(Y, dtype=np.float32)

def LstmCell():
    lstm_cell = rnn.BasicLSTMCell(HIDDEN_SIZE,state_is_tuple=True)
    return lstm_cell

# 定义lstm模型
def lstm_model(X, y):
    cell = rnn.MultiRNNCell([LstmCell() for _ in range(NUM_LAYERS)])
    output, _ = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    output = tf.reshape(output, [-1, HIDDEN_SIZE])
    # 通过无激活函数的全连接层计算线性回归，并将数据压缩成一维数组结构
    predictions = tf.contrib.layers.fully_connected(output, 1, None)
    
    # 将predictions和labels调整统一的shape
    labels = tf.reshape(y, [-1])
    predictions = tf.reshape(predictions, [-1])
    
    loss = tf.losses.mean_squared_error(predictions, labels)
    train_op = tf.contrib.layers.optimize_loss(loss, tf.train.get_global_step(),
                                             optimizer="Adagrad",
                                             learning_rate=0.1)
    return predictions, loss, train_op

# 进行训练
# 封装之前定义的lstm
regressor = SKCompat(learn.Estimator(model_fn=lstm_model, model_dir="Models/model_2"))
# 生成数据
test_start = TRAINING_EXAMPLES * SAMPLE_GAP
test_end = (TRAINING_EXAMPLES + TESTING_EXAMPLES) * SAMPLE_GAP
train_X, train_y = generate_data(np.sin(np.linspace(0, test_start, TRAINING_EXAMPLES, dtype=np.float32)))
test_X, test_y = generate_data(np.sin(np.linspace(test_start, test_end, TESTING_EXAMPLES, dtype=np.float32)))
# 拟合数据
regressor.fit(train_X, train_y, batch_size=BATCH_SIZE, steps=TRAINING_STEPS)
# 计算预测值
predicted = [[pred] for pred in regressor.predict(test_X)]

# 计算MSE
rmse = np.sqrt(((predicted - test_y) ** 2).mean(axis=0))
print("Mean Square Error is:%f" % rmse[0])

plot_predicted, = plt.plot(predicted, label='predicted')
plot_test, = plt.plot(test_y, label='real_sin')
plt.legend([plot_predicted, plot_test],['predicted', 'real_sin'])
plt.show()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1180c4210>, '_model_dir': 'Models/model_2', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_save_summary_steps': 100, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_evaluation_master': '', '_master': ''}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from Models/model_2/model.ckpt-40000
INFO:tensorflow:Saving checkpoints for 40001 into Models/model_2/model.ckpt.
INFO:tensorflow:loss = 2.47379e-06, step = 40001
INFO:tensorflow:global_step/sec: 331.374
INFO:tensorflow:loss = 2.7048e-06, step = 4

INFO:tensorflow:global_step/sec: 355.991
INFO:tensorflow:loss = 3.07711e-06, step = 47301 (0.281 sec)
INFO:tensorflow:global_step/sec: 408.423
INFO:tensorflow:loss = 2.82055e-06, step = 47401 (0.250 sec)
INFO:tensorflow:global_step/sec: 315.743
INFO:tensorflow:loss = 3.03083e-06, step = 47501 (0.315 sec)
INFO:tensorflow:global_step/sec: 263.963
INFO:tensorflow:loss = 2.46231e-06, step = 47601 (0.376 sec)
INFO:tensorflow:global_step/sec: 384.991
INFO:tensorflow:loss = 3.72102e-06, step = 47701 (0.261 sec)
INFO:tensorflow:global_step/sec: 336.719
INFO:tensorflow:loss = 4.35092e-06, step = 47801 (0.298 sec)
INFO:tensorflow:global_step/sec: 241.751
INFO:tensorflow:loss = 3.29385e-06, step = 47901 (0.412 sec)
INFO:tensorflow:global_step/sec: 471.404
INFO:tensorflow:loss = 4.04725e-06, step = 48001 (0.213 sec)
INFO:tensorflow:global_step/sec: 273.049
INFO:tensorflow:loss = 3.90292e-06, step = 48101 (0.366 sec)
INFO:tensorflow:global_step/sec: 314.282
INFO:tensorflow:loss = 2.85937e-06, step 

AttributeError: 'Legend' object has no attribute 'savefig'

In [29]:
plot_predicted, = plt.plot(predicted, label='predicted')
# plot_test, = plt.plot(test_y, label='real_sin')
# plt.legend([plot_predicted, plot_test],['predicted', 'real_sin'])
plt.show()

In [24]:
# plot_predicted, = plt.plot(predicted, label='predicted')
# plot_test, = plt.plot(test_y, label='real_sin')
# plt.legend([plot_predicted, plot_test],['predicted', 'real_sin'])

pic01 = plt.figure(figsize=(19, 12))
plt.plot([i for i in range(len(predicted))], [i[0] for i in predicted], 'ro')
plt.plot([i for i in range(len(test_y))], [i[0] for i in test_y], 'bo')
plt.show()
pic01.savefig('temp.png') 

In [21]:
predicted

[[-0.41675103],
 [-0.40750104],
 [-0.39821219],
 [-0.38888556],
 [-0.37952238],
 [-0.37012362],
 [-0.36069065],
 [-0.35122108],
 [-0.3417207],
 [-0.3321898],
 [-0.32262927],
 [-0.31304049],
 [-0.30342507],
 [-0.29378444],
 [-0.28411835],
 [-0.27442807],
 [-0.26471478],
 [-0.2549786],
 [-0.24522142],
 [-0.23544435],
 [-0.22564851],
 [-0.21583484],
 [-0.20600449],
 [-0.19615845],
 [-0.18629773],
 [-0.17642328],
 [-0.16653621],
 [-0.15663749],
 [-0.14672807],
 [-0.13680895],
 [-0.12688111],
 [-0.1169454],
 [-0.1070029],
 [-0.097054452],
 [-0.087100998],
 [-0.077143416],
 [-0.06718269],
 [-0.057219625],
 [-0.047251597],
 [-0.03728421],
 [-0.027317449],
 [-0.017352149],
 [-0.007389456],
 [0.0025694743],
 [0.012523517],
 [0.022473127],
 [0.032417279],
 [0.042355161],
 [0.052287094],
 [0.062211283],
 [0.072126895],
 [0.082033128],
 [0.091929197],
 [0.10181433],
 [0.11168772],
 [0.12154859],
 [0.13139614],
 [0.14122961],
 [0.15104821],
 [0.16085112],
 [0.17063761],
 [0.18040684],
 [0.19015807]

In [13]:
print len(test_y)
print len(predicted)

989
989
